In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Pre processing 

In [78]:
df = pd.read_csv('./bike-sharing-demand/train.csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [79]:
df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [80]:
df = df.drop(['datetime','casual','registered'],1)
df.head()
X = df.drop(['count'],1)
print(np.shape(X))
y= df['count'].values

(10886, 8)


# Feature selection

# Using Correlation

In [81]:
co_matrix = df.corr()
co_matrix 

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
season,1.000000,0.029368,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.163439
holiday,0.029368,1.000000,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,-0.005393
workingday,-0.008126,-0.250491,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,0.011594
weather,0.008879,-0.007074,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.128655
temp,0.258689,0.000295,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.394454
atemp,0.264744,-0.005215,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.389784
humidity,0.190610,0.001929,-0.010880,0.406244,-0.064949,-0.043536,1.000000,-0.318607,-0.317371
windspeed,-0.147121,0.008409,0.013373,0.007261,-0.017852,-0.057473,-0.318607,1.000000,0.101369
count,0.163439,-0.005393,0.011594,-0.128655,0.394454,0.389784,-0.317371,0.101369,1.000000


In [82]:
co_matrix [np.abs(co_matrix) < .5]= 0
co_matrix

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
season,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
holiday,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
workingday,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0
weather,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0
temp,0.0,0.0,0.0,0.0,1.000000,0.984948,0.0,0.0,0.0
atemp,0.0,0.0,0.0,0.0,0.984948,1.000000,0.0,0.0,0.0
humidity,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0
windspeed,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0
count,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0


    SELECTED: casual	registered

# Detecting Collinearity with Eigenvectors

       Note that index 10, eigenvalue of -5.381393e-15, is near to zero or very small compared to the others. Small value  represents presence of collinearity.
       
       

In [83]:
eigenvalues, eigenvectors = np.linalg.eig(df.corr())

In [84]:
pd.Series(eigenvalues).sort_values()

2    0.014165
4    0.419298
7    0.669269
8    0.746907
6    0.822449
5    1.018323
3    1.257094
1    1.661675
0    2.390821
dtype: float64

In [85]:
np.abs(pd.Series(eigenvectors[:,8])).sort_values(ascending=False)


2    0.697325
1    0.694089
8    0.127328
3    0.082067
7    0.059804
0    0.053779
4    0.034820
5    0.027470
6    0.024373
dtype: float64

print("SELECTED:")
print(df.columns[8], df.columns[2], df.columns[9], df.columns[6])

In [105]:
print("SELECTED:") 
print( df.columns[2], df.columns[1], df.columns[3])

eigen_column =[df.columns[2], df.columns[1], df.columns[3]]

SELECTED:
workingday holiday weather


### Standardise Variable to Identify Key Feature(s)
In order to perform point Impact of the variable / factor on the model properly, one needs to standardise the variable

and from list we need to take the leargest 


## Before StandardScaler  

In [87]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X,y)
result = pd.DataFrame(list(zip(model.coef_, df.columns)), columns=['coefficient', 'name']).set_index('name')
np.abs(result).sort_values(by='coefficient', ascending=False)

,coefficient
name,
season,22.758820
holiday,9.158724
weather,5.938718
atemp,5.631203
humidity,3.056835
temp,1.847368
workingday,1.149531
windspeed,0.777616


## After StandardScaler  

In [88]:
from sklearn.preprocessing import StandardScaler  
from sklearn.pipeline import make_pipeline  
scaler = StandardScaler()  
standard_coefficient_linear_reg = make_pipeline(scaler, model)

In [89]:
standard_coefficient_linear_reg.fit(X,y)
result = pd.DataFrame(list(zip(standard_coefficient_linear_reg.steps[1][1].coef_, df.columns)), 
                      columns=['coefficient', 'name']).set_index('name')
np.abs(result).sort_values(by='coefficient', ascending=False)

,coefficient
name,
humidity,58.826192
atemp,47.720001
season,25.401644
temp,14.393276
windspeed,6.348584
weather,3.764016
holiday,1.525763
workingday,0.535840


# Train Model

## Using StandardScaler

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

np.shape(X)

(10886, 8)

In [91]:
standard_coefficient_linear_reg.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [92]:
from sklearn.metrics import mean_squared_error
y_pred = standard_coefficient_linear_reg.predict( X_test)
mean_squared_error(y_test,y_pred )

23823.835509160603

## Using Eigenvalues

In [107]:
X = df[eigen_column]
 
y = df['count'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [109]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [110]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict( X_test)
mean_squared_error(y_test,y_pred )

31732.957254931272

## Using Correlation

In [54]:
X = df[['casual' ,'workingday' ,'registered' ,'humidity']]
y = df['count'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

from sklearn.metrics import mean_squared_error
y_pred = model.predict( X_test)
mean_squared_error(y_test,y_pred )

1.5346031218036115e-26

# 1st model is our best model

In [111]:
df1 = pd.read_csv('./bike-sharing-demand/test.csv')
df1.head()


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [112]:
X_TEST = df1.drop(['datetime'],1)
np.shape(X_TEST)

(6493, 8)

In [114]:
res = standard_coefficient_linear_reg.predict(X_TEST)

In [116]:
out = pd.DataFrame()
out['output'] = res
out.to_csv('./bike-sharing-demand/output.csv')

In [117]:
min(res)

-52.43597817740252